# 02 - Veri Temizleme ve Birleştirme

Bu notebook, İbrahim'in topladığı ham verileri temizleyip birleştirerek model eğitimi için hazır `master_merged.csv` dosyasını oluşturur.

## Görevler:
1. Ham maç verisini temizleme (NocturneBear 2010-2024)
2. Tarih formatlarını normalize etme
3. Takım isimlerini standartlaştırma
4. Takım istatistiklerini birleştirme
5. Çıktı dosyalarını oluşturma:
   - `long_term_2010_2025.csv`
   - `season_2025_current.csv`
   - `master_merged.csv`


## 1. Import ve Setup


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Proje root dizinini bul (notebooks/ klasöründen bir üst dizin)
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Çalışma dizinini proje root'una ayarla
os.chdir(project_root)

# clean_merge modülünü import et
from src.data.clean_merge import (
    normalize_date,
    standardize_team_names,
    clean_game_data,
    merge_nocturnebear_data,
    merge_team_stats,
    create_master_merged
)

print("Kütüphaneler yüklendi!")
print(f"Çalışma dizini: {os.getcwd()}")


Kütüphaneler yüklendi!
Çalışma dizini: c:\Users\user\Desktop\ann odev


## 2. Veri Yükleme ve İlk İnceleme


In [2]:
# Klasör yollarını tanımla (proje root'undan)
data_raw_path = project_root / 'data_raw'
nocturnebear_path = data_raw_path / 'nocturnebear_2010_2024'
team_stats_path = data_raw_path / 'nbastuffer_2025_2026_team_stats_raw.csv'
bdl_games_path = data_raw_path / 'bdl_games_2025.csv'  # Varsa

print(f"NocturneBear klasörü: {nocturnebear_path.exists()}")
print(f"Takım istatistikleri: {team_stats_path.exists()}")
print(f"BDL games 2025: {bdl_games_path.exists()}")
print(f"\nProje root: {project_root}")
print(f"Data raw path: {data_raw_path}")


NocturneBear klasörü: True
Takım istatistikleri: True
BDL games 2025: False

Proje root: c:\Users\user\Desktop\ann odev
Data raw path: c:\Users\user\Desktop\ann odev\data_raw


In [3]:
# NocturneBear klasöründeki dosyaları listele
if nocturnebear_path.exists():
    files = list(nocturnebear_path.glob('*.csv'))
    print(f"\nNocturneBear klasöründe {len(files)} CSV dosyası bulundu:")
    for f in files:
        print(f"  - {f.name}")
else:
    print("\n⚠ NocturneBear klasörü bulunamadı!")



NocturneBear klasöründe 6 CSV dosyası bulundu:
  - play_off_box_scores_2010_2024.csv
  - play_off_totals_2010_2024.csv
  - regular_season_box_scores_2010_2024_part_1.csv
  - regular_season_box_scores_2010_2024_part_2.csv
  - regular_season_box_scores_2010_2024_part_3.csv
  - regular_season_totals_2010_2024.csv


## 3. NocturneBear Verilerini Yükleme ve Birleştirme


In [4]:
# NocturneBear verilerini yükle ve birleştir (maç bazlı formata dönüştür)
try:
    nocturnebear_df = merge_nocturnebear_data(nocturnebear_path, convert_to_game_level=True)
    print(f"\n✓ Toplam {len(nocturnebear_df)} maç yüklendi (box score'dan dönüştürüldü)")
    print(f"✓ Toplam {len(nocturnebear_df.columns)} sütun")
except Exception as e:
    print(f"\n❌ Hata: {e}")
    import traceback
    traceback.print_exc()
    nocturnebear_df = None


Yükleniyor: regular_season_box_scores_2010_2024_part_1.csv
Yükleniyor: regular_season_box_scores_2010_2024_part_2.csv
Yükleniyor: regular_season_box_scores_2010_2024_part_3.csv
Yükleniyor: play_off_box_scores_2010_2024.csv
DataFrame'ler birleştiriliyor...
Toplam 455663 satır birleştirildi (box score formatında)
Sütunlar: ['season_year', 'game_date', 'gameId', 'matchup', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId', 'personName', 'position', 'comment', 'jerseyNum', 'minutes', 'fieldGoalsMade', 'fieldGoalsAttempted', 'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted', 'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted', 'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive', 'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers', 'foulsPersonal', 'points', 'plusMinusPoints', 'season_type']

Box score verisi maç bazlı formata dönüştürülüyor...
Box score'dan 17828 maç oluşturuldu (orijinal 455663 satırdan)
Maç bazl

In [5]:
# İlk birkaç satırı incele
if nocturnebear_df is not None:
    print("\nİlk 5 satır:")
    display(nocturnebear_df.head())
    
    print("\nSütun isimleri:")
    print(list(nocturnebear_df.columns))
    
    print("\nVeri tipleri:")
    display(nocturnebear_df.dtypes)
    
    print("\nEksik değer özeti:")
    missing = nocturnebear_df.isnull().sum()
    missing_pct = (missing / len(nocturnebear_df)) * 100
    missing_df = pd.DataFrame({
        'Eksik Sayı': missing,
        'Yüzde': missing_pct
    })
    display(missing_df[missing_df['Eksik Sayı'] > 0].sort_values('Eksik Sayı', ascending=False))



İlk 5 satır:


,gameId,game_date,season_year,home_team,away_team,home_team_code,away_team_code,home_score,away_score,season_type,matchup
0,21000112,2010-11-10,2010-11,Cleveland Cavaliers,Brooklyn Nets,CLE,NJN,87.0,95.0,regular,NJN @ CLE
1,21000165,2010-11-17,2010-11,Utah Jazz,Brooklyn Nets,UTA,NJN,98.0,88.0,regular,NJN @ UTA
2,21000237,2010-11-27,2010-11,Philadelphia 76ers,Brooklyn Nets,PHI,NJN,102.0,86.0,regular,NJN @ PHI
3,21000351,2010-12-12,2010-11,Brooklyn Nets,Los Angeles Lakers,NJN,LAL,92.0,99.0,regular,NJN vs. LAL
4,21000020,2010-10-29,2010-11,Brooklyn Nets,Sacramento Kings,NJN,SAC,106.0,100.0,regular,NJN vs. SAC



Sütun isimleri:
['gameId', 'game_date', 'season_year', 'home_team', 'away_team', 'home_team_code', 'away_team_code', 'home_score', 'away_score', 'season_type', 'matchup']

Veri tipleri:


gameId              int64
game_date          object
season_year        object
home_team          object
away_team          object
home_team_code     object
away_team_code     object
home_score        float64
away_score        float64
season_type        object
matchup            object
dtype: object


Eksik değer özeti:


,Eksik Sayı,Yüzde


## 4. Tarih Normalizasyonu


In [6]:
# Tarih sütununu bul ve normalize et
if nocturnebear_df is not None:
    # Maç bazlı formata dönüştürüldüyse game_date kullan
    if 'game_date' in nocturnebear_df.columns:
        date_col = 'game_date'
    else:
        date_cols = [col for col in nocturnebear_df.columns if 'date' in col.lower()]
        date_col = date_cols[0] if date_cols else None
    
    if date_col:
        print(f"Tarih sütunu bulundu: '{date_col}'")
        
        print(f"\nOrijinal tarih formatı örnekleri:")
        print(nocturnebear_df[date_col].head(10))
        
        # Normalize et
        nocturnebear_df[date_col] = normalize_date(nocturnebear_df[date_col])
        
        print(f"\nNormalize edilmiş tarih formatı:")
        print(nocturnebear_df[date_col].head(10))
        
        print(f"\nTarih aralığı:")
        print(f"  En erken: {nocturnebear_df[date_col].min()}")
        print(f"  En geç: {nocturnebear_df[date_col].max()}")
    else:
        print("⚠ Tarih sütunu bulunamadı!")
        date_col = None


Tarih sütunu bulundu: 'game_date'

Orijinal tarih formatı örnekleri:
0    2010-11-10
1    2010-11-17
2    2010-11-27
3    2010-12-12
4    2010-10-29
5    2010-12-14
6    2010-10-31
7    2010-10-27
8    2010-11-19
9    2010-11-24
Name: game_date, dtype: object

Normalize edilmiş tarih formatı:
0   2010-11-10
1   2010-11-17
2   2010-11-27
3   2010-12-12
4   2010-10-29
5   2010-12-14
6   2010-10-31
7   2010-10-27
8   2010-11-19
9   2010-11-24
Name: game_date, dtype: datetime64[ns]

Tarih aralığı:
  En erken: 2010-10-26 00:00:00
  En geç: 2024-06-17 00:00:00


## 5. Takım İsmi Standartlaştırma


In [7]:
# Takım isimlerini standartlaştır (maç bazlı formatta home_team ve away_team var)
if nocturnebear_df is not None:
    print("Takım isimleri standartlaştırılıyor...")
    
    # Maç bazlı formatta home_team ve away_team sütunları olmalı
    if 'home_team' in nocturnebear_df.columns and 'away_team' in nocturnebear_df.columns:
        print("\nTakım sütunları: home_team, away_team")
        
        # Standartlaştırmadan önce örnekler
        print("\nStandartlaştırmadan önce örnek takım isimleri:")
        print("\nhome_team:")
        print(nocturnebear_df['home_team'].value_counts().head(10))
        print("\naway_team:")
        print(nocturnebear_df['away_team'].value_counts().head(10))
        
        # Standartlaştır
        nocturnebear_df = standardize_team_names(nocturnebear_df, ['home_team', 'away_team'])
        
        print("\n✓ Takım isimleri standartlaştırıldı")
        print("\nStandartlaştırmadan sonra örnek takım isimleri:")
        print("\nhome_team:")
        print(nocturnebear_df['home_team'].value_counts().head(10))
        print("\naway_team:")
        print(nocturnebear_df['away_team'].value_counts().head(10))
    else:
        # Eski format (box score) için
        team_cols = [col for col in nocturnebear_df.columns 
                     if 'team' in col.lower() or 'home' in col.lower() or 'away' in col.lower()]
        if team_cols:
            nocturnebear_df = standardize_team_names(nocturnebear_df)
            print("✓ Takım isimleri standartlaştırıldı")
        else:
            print("⚠ Takım sütunu bulunamadı!")


Takım isimleri standartlaştırılıyor...

Takım sütunları: home_team, away_team

Standartlaştırmadan önce örnek takım isimleri:

home_team:
home_team
Miami Heat               729
San Antonio Spurs        647
Houston Rockets          620
Atlanta Hawks            616
Milwaukee Bucks          614
Philadelphia 76ers       611
Los Angeles Lakers       611
Utah Jazz                610
Boston Celtics           609
Oklahoma City Thunder    609
Name: count, dtype: int64

away_team:
away_team
Boston Celtics            678
Golden State Warriors     658
Cleveland Cavaliers       642
Memphis Grizzlies         631
Dallas Mavericks          630
Portland Trail Blazers    626
Oklahoma City Thunder     621
Toronto Raptors           618
LA Clippers               616
Indiana Pacers            613
Name: count, dtype: int64

✓ Takım isimleri standartlaştırıldı

Standartlaştırmadan sonra örnek takım isimleri:

home_team:
home_team
Miami Heat               729
San Antonio Spurs        647
Houston Rockets       

## 6. Veri Temizleme


In [8]:
# Veriyi temizle (maç bazlı formatta home_score ve away_score ile)
if nocturnebear_df is not None:
    original_len = len(nocturnebear_df)
    print(f"Orijinal satır sayısı: {original_len}")
    
    # Temizle (maç bazlı formatta score_cols otomatik tespit edilecek)
    nocturnebear_df = clean_game_data(
        nocturnebear_df, 
        score_cols=['home_score', 'away_score'] if 'home_score' in nocturnebear_df.columns else None,
        date_col=date_col if 'date_col' in locals() and date_col else 'game_date'
    )
    
    cleaned_len = len(nocturnebear_df)
    print(f"\nTemizlenmiş satır sayısı: {cleaned_len}")
    print(f"Kaldırılan satır sayısı: {original_len - cleaned_len}")
    
    print("\n✓ Veri temizlendi")


Orijinal satır sayısı: 17828
Temizleme: 8 satır kaldırıldı (17828 -> 17820)

Temizlenmiş satır sayısı: 17820
Kaldırılan satır sayısı: 8

✓ Veri temizlendi


## 7. BDL Games 2025 Verilerini Ekleme (Varsa)


In [9]:
# BDL games 2025 varsa yükle ve ekle
if bdl_games_path.exists() and nocturnebear_df is not None:
    print("BDL Games 2025 verileri yükleniyor...")
    
    try:
        bdl_df = pd.read_csv(bdl_games_path, low_memory=False)
        bdl_df['season_type'] = 'regular'
        
        # Tarih normalizasyonu
        if 'date_col' in locals() and date_col:
            bdl_date_cols = [col for col in bdl_df.columns if 'date' in col.lower()]
            if bdl_date_cols:
                bdl_df[bdl_date_cols[0]] = normalize_date(bdl_df[bdl_date_cols[0]])
        
        # Takım isimlerini standartlaştır
        bdl_df = standardize_team_names(bdl_df)
        
        # Temizle
        bdl_df = clean_game_data(bdl_df, date_col=date_col if 'date_col' in locals() and date_col else 'date')
        
        # Birleştir
        nocturnebear_df = pd.concat([nocturnebear_df, bdl_df], ignore_index=True)
        
        print(f"✓ BDL verileri eklendi. Toplam satır: {len(nocturnebear_df)}")
    except Exception as e:
        print(f"⚠ BDL verileri yüklenirken hata: {e}")
else:
    print("BDL Games 2025 dosyası bulunamadı veya NocturneBear verisi yok. Atlanıyor...")


BDL Games 2025 dosyası bulunamadı veya NocturneBear verisi yok. Atlanıyor...


## 8. Takım İstatistiklerini Birleştirme


In [10]:
# Takım istatistiklerini yükle ve birleştir
if team_stats_path.exists() and nocturnebear_df is not None:
    print("Takım istatistikleri yükleniyor...")
    
    try:
        team_stats_df = pd.read_csv(team_stats_path, low_memory=False)
        
        print(f"\nTakım istatistikleri:")
        print(f"  Satır sayısı: {len(team_stats_df)}")
        print(f"  Sütun sayısı: {len(team_stats_df.columns)}")
        print(f"\nSütunlar: {list(team_stats_df.columns)}")
        
        # Takım isimlerini standartlaştır (TEAM sütunu için)
        team_stats_df = standardize_team_names(team_stats_df, ['TEAM'])
        
        print(f"\nTakım istatistikleri standartlaştırıldı:")
        print(f"  Örnek takım isimleri: {list(team_stats_df['TEAM'].unique()[:5])}")
        
        # Birleştir (maç bazlı veri ile)
        nocturnebear_df = merge_team_stats(
            nocturnebear_df,
            team_stats_df,
            date_col=date_col if 'date_col' in locals() and date_col else 'game_date',
            home_team_col='home_team',
            away_team_col='away_team',
            team_col_in_stats='TEAM'
        )
        
        print(f"\n✓ Takım istatistikleri birleştirildi")
        print(f"  Yeni toplam sütun sayısı: {len(nocturnebear_df.columns)}")
        
        # Merge başarı oranını kontrol et
        home_stats_cols = [col for col in nocturnebear_df.columns if col.startswith('home_team_')]
        away_stats_cols = [col for col in nocturnebear_df.columns if col.startswith('away_team_')]
        print(f"  Home team istatistik sütunları: {len(home_stats_cols)}")
        print(f"  Away team istatistik sütunları: {len(away_stats_cols)}")
        
        if len(home_stats_cols) > 0:
            # Merge başarı oranı
            merged_count = nocturnebear_df[home_stats_cols[0]].notna().sum()
            total_count = len(nocturnebear_df)
            print(f"  Merge başarı oranı: {merged_count}/{total_count} ({merged_count/total_count*100:.1f}%)")
    except Exception as e:
        print(f"⚠ Takım istatistikleri birleştirilirken hata: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Takım istatistikleri dosyası bulunamadı veya NocturneBear verisi yok. Atlanıyor...")


Takım istatistikleri yükleniyor...

Takım istatistikleri:
  Satır sayısı: 30
  Sütun sayısı: 24

Sütunlar: ['RANK', 'TEAM', 'CONF', 'DIVISION', 'GP', 'PPG', 'oPPG', 'pDIFF', 'PACE', 'oEFF', 'dEFF', 'eDIFF', 'SoS', 'rSoS', 'SAR', 'CONS', 'A4F', 'W', 'L', 'WIN%', 'eWIN%', 'pWIN%', 'ACH', 'STRK']

Takım istatistikleri standartlaştırıldı:
  Örnek takım isimleri: ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls']
⚠ Takım istatistikleri birleştirilirken hata: merge_team_stats() got an unexpected keyword argument 'date_col'


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_3864\3054917194.py", line 20, in <module>
    nocturnebear_df = merge_team_stats(
        nocturnebear_df,
    ...<4 lines>...
        team_col_in_stats='TEAM'
    )
TypeError: merge_team_stats() got an unexpected keyword argument 'date_col'


## 9. Tarihe Göre Sıralama


In [11]:
# Tarihe göre sırala
if nocturnebear_df is not None:
    # date_col'u belirle
    if 'date_col' in locals() and date_col:
        sort_col = date_col
    elif 'game_date' in nocturnebear_df.columns:
        sort_col = 'game_date'
    else:
        sort_col = None
    
    if sort_col and sort_col in nocturnebear_df.columns:
        nocturnebear_df = nocturnebear_df.sort_values(by=sort_col).reset_index(drop=True)
        print(f"✓ Veriler tarihe göre sıralandı")
        print(f"  En erken tarih: {nocturnebear_df[sort_col].min()}")
        print(f"  En geç tarih: {nocturnebear_df[sort_col].max()}")


✓ Veriler tarihe göre sıralandı
  En erken tarih: 2010-10-26 00:00:00
  En geç tarih: 2024-06-17 00:00:00


## 10. Çıktı Dosyalarını Oluşturma


In [12]:
# Çıktı klasörünü oluştur (proje root'undan)
output_dir = project_root / 'data_processed'
output_dir.mkdir(exist_ok=True)

print(f"Çıktı klasörü: {output_dir.absolute()}")


Çıktı klasörü: c:\Users\user\Desktop\ann odev\data_processed


In [13]:
# 10.1 long_term_2010_2025.csv
if nocturnebear_df is not None:
    long_term_df = nocturnebear_df.copy()
    long_term_path = output_dir / 'long_term_2010_2025.csv'
    
    long_term_df.to_csv(long_term_path, index=False)
    print(f"✓ long_term_2010_2025.csv kaydedildi")
    print(f"  Dosya yolu: {long_term_path.absolute()}")
    print(f"  Satır sayısı: {len(long_term_df)}")
    print(f"  Sütun sayısı: {len(long_term_df.columns)}")


✓ long_term_2010_2025.csv kaydedildi
  Dosya yolu: c:\Users\user\Desktop\ann odev\data_processed\long_term_2010_2025.csv
  Satır sayısı: 17820
  Sütun sayısı: 11


In [14]:
# 10.2 season_2025_current.csv (sadece 2025 sezonu)
if nocturnebear_df is not None:
    # Tarih sütununu belirle
    if 'date_col' in locals() and date_col and date_col in nocturnebear_df.columns:
        date_col_for_filter = date_col
    elif 'game_date' in nocturnebear_df.columns:
        date_col_for_filter = 'game_date'
    else:
        date_col_for_filter = None
    
    if date_col_for_filter:
        season_2025_df = nocturnebear_df[
            nocturnebear_df[date_col_for_filter] >= pd.Timestamp('2025-01-01')
        ].copy()
        
        if len(season_2025_df) > 0:
            season_2025_path = output_dir / 'season_2025_current.csv'
            season_2025_df.to_csv(season_2025_path, index=False)
            print(f"✓ season_2025_current.csv kaydedildi")
            print(f"  Dosya yolu: {season_2025_path.absolute()}")
            print(f"  Satır sayısı: {len(season_2025_df)}")
            print(f"  Sütun sayısı: {len(season_2025_df.columns)}")
        else:
            print("⚠ 2025 sezonu verisi bulunamadı (normal - NocturneBear verisi 2010-2024)")
    else:
        print("⚠ Tarih sütunu bulunamadı")


⚠ 2025 sezonu verisi bulunamadı (normal - NocturneBear verisi 2010-2024)


In [15]:
# 10.3 master_merged.csv
if nocturnebear_df is not None:
    master_df = nocturnebear_df.copy()
    master_path = output_dir / 'master_merged.csv'
    
    master_df.to_csv(master_path, index=False)
    print(f"✓ master_merged.csv kaydedildi")
    print(f"  Dosya yolu: {master_path.absolute()}")
    print(f"  Satır sayısı: {len(master_df)}")
    print(f"  Sütun sayısı: {len(master_df.columns)}")
    
    print("\n" + "="*60)
    print("ÖZET")
    print("="*60)
    print(f"Toplam maç sayısı: {len(master_df)}")
    
    # Tarih sütununu belirle
    if 'date_col' in locals() and date_col and date_col in master_df.columns:
        date_col_for_summary = date_col
    elif 'game_date' in master_df.columns:
        date_col_for_summary = 'game_date'
    else:
        date_col_for_summary = None
    
    if date_col_for_summary:
        print(f"Tarih aralığı: {master_df[date_col_for_summary].min()} - {master_df[date_col_for_summary].max()}")
    
    print(f"Toplam özellik sayısı: {len(master_df.columns)}")
    
    # Takım istatistikleri kontrolü
    home_stats_cols = [col for col in master_df.columns if col.startswith('home_team_')]
    away_stats_cols = [col for col in master_df.columns if col.startswith('away_team_')]
    if home_stats_cols:
        print(f"Takım istatistikleri: {len(home_stats_cols)} home + {len(away_stats_cols)} away sütun")
        merged_count = master_df[home_stats_cols[0]].notna().sum()
        print(f"Takım istatistikleri merge oranı: {merged_count}/{len(master_df)} ({merged_count/len(master_df)*100:.1f}%)")
    
    print("\n✓ Tüm çıktı dosyaları başarıyla oluşturuldu!")


✓ master_merged.csv kaydedildi
  Dosya yolu: c:\Users\user\Desktop\ann odev\data_processed\master_merged.csv
  Satır sayısı: 17820
  Sütun sayısı: 11

ÖZET
Toplam maç sayısı: 17820
Tarih aralığı: 2010-10-26 00:00:00 - 2024-06-17 00:00:00
Toplam özellik sayısı: 11
Takım istatistikleri: 1 home + 1 away sütun
Takım istatistikleri merge oranı: 17820/17820 (100.0%)

✓ Tüm çıktı dosyaları başarıyla oluşturuldu!


## 11. Tüm Verileri Birleştirme (ÖNERİLEN YÖNTEM)

**ÖNEMLİ:** Yukarıdaki adımlar sadece temel merge işlemlerini yapıyor. Tüm özellikleri (rest days stats, schedule rest days, gamelog 2025) içeren tam birleştirme için aşağıdaki `create_master_merged` fonksiyonunu kullanın.

Bu fonksiyon şunları yapar:
- NocturneBear verilerini yükler ve maç bazlı formata dönüştürür
- Gamelog'dan 2025 verilerini çıkarır
- Takım istatistiklerini birleştirir
- Rest days stats'i birleştirir
- Schedule rest days'i birleştirir
- Tüm çıktı dosyalarını oluşturur (`master_merged.csv`, `long_term_2010_2025.csv`, `season_2025_current.csv`)


In [16]:
# Tüm işlemi tek seferde yap (ÖNERİLEN YÖNTEM)
# Bu fonksiyon tüm özellikleri içerir:
# - NocturneBear verileri (2010-2024)
# - Gamelog'dan 2025 verileri
# - Takım istatistikleri
# - Rest days stats
# - Schedule rest days

results = create_master_merged(
    output_dir='data_processed/',
    nocturnebear_path='data_raw/nocturnebear_2010_2024/',
    team_stats_path='data_raw/nbastuffer_2025_2026_team_stats_raw.csv',
    bdl_games_path='data_raw/bdl_games_2025.csv' if Path('data_raw/bdl_games_2025.csv').exists() else None
)

print("\n" + "="*60)
print("ÇIKTI DOSYALARI")
print("="*60)
for key, df in results.items():
    if isinstance(df, pd.DataFrame):
        print(f"  {key}: {len(df)} satır, {len(df.columns)} sütun")

# Master merged dosyasını kontrol et
if 'master' in results:
    master_df = results['master']
    print("\n" + "="*60)
    print("MASTER_MERGED.CSV ÖZET")
    print("="*60)
    print(f"Toplam maç sayısı: {len(master_df)}")
    
    if 'game_date' in master_df.columns:
        print(f"Tarih aralığı: {master_df['game_date'].min()} - {master_df['game_date'].max()}")
    
    print(f"Toplam özellik sayısı: {len(master_df.columns)}")
    
    # Takım istatistikleri kontrolü
    home_stats_cols = [col for col in master_df.columns if col.startswith('home_team_')]
    away_stats_cols = [col for col in master_df.columns if col.startswith('away_team_')]
    if home_stats_cols:
        print(f"Takım istatistikleri: {len(home_stats_cols)} home + {len(away_stats_cols)} away sütun")
        merged_count = master_df[home_stats_cols[0]].notna().sum()
        print(f"Takım istatistikleri merge oranı: {merged_count}/{len(master_df)} ({merged_count/len(master_df)*100:.1f}%)")
    
    # Rest days stats kontrolü
    home_rest_cols = [col for col in master_df.columns if col.startswith('home_rest_')]
    away_rest_cols = [col for col in master_df.columns if col.startswith('away_rest_')]
    if home_rest_cols:
        print(f"Rest days istatistikleri: {len(home_rest_cols)} home + {len(away_rest_cols)} away sütun")
    
    # Schedule stats kontrolü
    home_schedule_cols = [col for col in master_df.columns if col.startswith('home_schedule_')]
    away_schedule_cols = [col for col in master_df.columns if col.startswith('away_schedule_')]
    if home_schedule_cols:
        print(f"Schedule rest days istatistikleri: {len(home_schedule_cols)} home + {len(away_schedule_cols)} away sütun")
    
    # Boş sütun kontrolü
    empty_cols = master_df.columns[master_df.isnull().all()].tolist()
    if empty_cols:
        print(f"\nBoş sütunlar (tamamen boş): {len(empty_cols)}")
        print(f"  {empty_cols[:5]}{'...' if len(empty_cols) > 5 else ''}")
    
    print("\n✓ Tüm çıktı dosyaları başarıyla oluşturuldu!")


1. NocturneBear Verilerini Yükleme ve Birleştirme
Yükleniyor: regular_season_box_scores_2010_2024_part_1.csv
Yükleniyor: regular_season_box_scores_2010_2024_part_2.csv
Yükleniyor: regular_season_box_scores_2010_2024_part_3.csv
Yükleniyor: play_off_box_scores_2010_2024.csv
DataFrame'ler birleştiriliyor...
Toplam 455663 satır birleştirildi (box score formatında)
Sütunlar: ['season_year', 'game_date', 'gameId', 'matchup', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId', 'personName', 'position', 'comment', 'jerseyNum', 'minutes', 'fieldGoalsMade', 'fieldGoalsAttempted', 'fieldGoalsPercentage', 'threePointersMade', 'threePointersAttempted', 'threePointersPercentage', 'freeThrowsMade', 'freeThrowsAttempted', 'freeThrowsPercentage', 'reboundsOffensive', 'reboundsDefensive', 'reboundsTotal', 'assists', 'steals', 'blocks', 'turnovers', 'foulsPersonal', 'points', 'plusMinusPoints', 'season_type']

Box score verisi maç bazlı formata dönüştürülüyor...
Box score'dan 17828 m